In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torch transformers peft datasets accelerate sentencepiece

##  Carga del Modelo Base y Tokenizador

In [ ]:
!huggingface-cli login

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import torch
import torch.nn as nn
from datasets import load_dataset

# Nombre del modelo ligero
model_name = "meta-llama/Llama-3.2-1B-Instruct"

# Cargar modelo base y tokenizador
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Mapea automáticamente a la GPU/CPU
    torch_dtype=torch.float16  # Usa half-precision para reducir memoria
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configurar tokenizador
tokenizer.pad_token = "<pad>"
model.config.pad_token_id = tokenizer.pad_token_id

##  Configuración de LoRA

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

# Configuración optimizada de LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,  # Dimensión reducida
    lora_alpha=8,  # Escalado para más precisión
    lora_dropout=0.1,  # Dropout ligero
    target_modules=["q_proj", "v_proj"],  # Limitar a módulos críticos
    bias="none"
)

# Aplicar LoRA al modelo
model = get_peft_model(model, peft_config)

## Carga de Datos desde JSON

In [ ]:
# Configurar explícitamente el token <pad> si no está definido correctamente
if tokenizer.pad_token_id is None or tokenizer.pad_token != "<pad>":
    tokenizer.add_special_tokens({'pad_token': '<pad>'})  # Añadir token de padding
    model.resize_token_embeddings(len(tokenizer))  # Ajustar el vocabulario del modelo

# Configurar el ID del token <pad>
model.config.pad_token_id = tokenizer.pad_token_id

# Validar la configuración
print(f"Pad token: {tokenizer.pad_token}")
print(f"Pad token ID: {tokenizer.pad_token_id}")

In [ ]:
# Función de preprocesamiento
def preprocess_function(examples):
    # Combinar "prompt" y "response"
    inputs = [f"{prompt} {response}" for prompt, response in zip(examples["prompt"], examples["response"])]
    model_inputs = tokenizer(
        inputs,
        padding="max_length",  # Rellenar con pad_token
        truncation=True,       # Cortar a longitud máxima
        max_length=256,        # Definir longitud máxima
        return_tensors="pt"    # Retornar tensores
    )
    model_inputs["labels"] = model_inputs["input_ids"].clone().detach()  # Etiquetas = input_ids
    return model_inputs

# Aplicar tokenización al dataset
tokenized_dataset = split_dataset.map(preprocess_function, batched=True)

## Entrenamiento

In [ ]:
from transformers import TrainingArguments, Trainer

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,  # Tamaño de batch mayor porque el modelo es ligero
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    gradient_accumulation_steps=8,
    fp16=True,  # Half-precision para reducir tiempo
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Configurar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

In [ ]:
trainer.train()

## Guardado del Modelo

In [ ]:
# Guardar el modelo ajustado
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

## Cargar el modelo y el tokenizador entrenados

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Ruta al modelo y tokenizador guardados
model_path = "/content/drive/MyDrive/fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Configurar tokenizador
tokenizer.pad_token = "<pad>"
model.config.pad_token_id = tokenizer.pad_token_id

## Probar con un ejemplo de entrada

In [ ]:
# Realizar una predicción de ejemplo
test_example = "Las condiciones del clima y el suelo son las siguientes: Nitrógeno: 52.0, Fósforo: 39.0, Potasio: 39.0, Temperatura: 32.30°C, Humedad: 61.14%, pH: 6.55, Lluvia: 245.62 mm. ¿Qué cultivo recomiendas sembrar y por qué?"

inputs = tokenizer(test_example, return_tensors="pt", padding=True, truncation=True, max_length=512)
outputs = model.generate(inputs["input_ids"], max_length=500, num_return_sequences=1, attention_mask=inputs["attention_mask"])

predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(predicted_text)

## Optimización para Uso Offline

In [ ]:
from transformers import BitsAndBytesConfig

# Configuración de cuantización para optimización en 8 bits
quant_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Usar 8 bits para optimizar la memoria
    llm_int8_threshold=6.0,  # Umbral para activación de cuantización
    llm_int8_enable_fp32_cpu_offload=True  # Habilitar offloading de FP32 en CPU
)

model = AutoModelForCausalLM.from_pretrained(
    "./fine_tuned_model", quantization_config=quant_config, device_map="auto"
)